In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from synthcity.utils.serialization import save_to_file, load_from_file

In [2]:
base_path = Path("/home/bcebere/Documents/datasets/ward")

train_static = pd.read_csv(base_path / "ward_static_train_data.csv.gz")
train_temporal = pd.read_csv(base_path / "ward_temporal_train_data_eav.csv.gz")

test_static = pd.read_csv(base_path / "ward_static_test_data.csv.gz")
test_temporal = pd.read_csv(base_path / "ward_temporal_test_data_eav.csv.gz")

In [3]:
import numpy as np
import pandas as pd

def process_temporal(in_static_data, in_temporal_data):
    all_temporal_data = []
    horizons = []
    
    for uid in in_static_data["id"].unique():
        local_temporal_idx = in_temporal_data["id"] == uid
        local_temporal = in_temporal_data[local_temporal_idx]
        columns = sorted(local_temporal["variable"].unique())
        times = sorted(local_temporal["time"].unique())

        temporal_data = pd.DataFrame([], columns = columns)


        for horizon in times:
            local_temporal_horizon_idx = local_temporal["time"] == horizon
            local_temporal_horizon = local_temporal[local_temporal_horizon_idx]

            horizon_data = pd.DataFrame(-1 * np.ones((1, len(columns))), columns = columns)

            proc = local_temporal_horizon[["variable", "value"]]
            proc.index = local_temporal_horizon["variable"]
            proc = proc.drop(columns = ["variable"])
            proc = proc.T.reset_index(drop = True)
            print(len(proc.columns), len(columns))
            horizon_data[proc.columns] = proc

            temporal_data = pd.concat([temporal_data, horizon_data], ignore_index = True)
        temporal_data.index = times

        for col in columns:
            if col not in temporal_data:
                temporal_data[col] = -1
        horizons.append(temporal_data["time"])
        all_temporal_data.append(temporal_data[columns])
        
    assert len(all_temporal_data) == len(in_static_data)

    return all_temporal_data, horizons

def eav_to_wide(df):
    """Transform EAV format to WIDE format.
    
    Args:
        - df: EAV format dataframe
        
    Returns:
        - df_wide: WIDE format dataframe.    
    """
    # Original data needs the following four column name in order.
    col_names = list(df.columns)
    assert col_names[0] == "id"
    assert col_names[1] == "time"
    assert col_names[2] == "variable"
    assert col_names[3] == "value"

    # Convert EAV format to WIDE format
    df_wide = pd.pivot_table(df, index=["id", "time"], columns="variable", values="value").reset_index(level=[0, 1])
    return df_wide


train_temporal_wide = eav_to_wide(train_temporal)
test_temporal_wide = eav_to_wide(test_temporal)

In [4]:
train_temporal_wide[train_temporal_wide["id"] == 1]

variable,id,time,Best Motor Response,Best Verbal Response,CHLORIDE,CREATINEINE,DBP,Eye Opening,GLUCLOSE,Glasgow Coma Scale Score,...,POTASSIUM,Pulse,Respiratory Rate,SBP,SODIUM,SpO2,TOTAL CO2,Temperature,UREA NITROGEN,WHITE BLOOD CELL COUNT
0,1,0.0,5.0,5.0,99.0,0.6,107.00,3.0,133.0,13.0,...,3.8,78.0,12.00,174.0,136.0,100.00,21.0,98.6,8.0,19.75
1,1,2.0,5.0,5.0,NaN,NaN,96.50,3.0,NaN,13.0,...,NaN,85.5,17.00,156.5,NaN,100.00,NaN,98.1,NaN,NaN
2,1,3.0,5.0,5.0,NaN,NaN,104.00,3.0,NaN,13.0,...,NaN,79.0,18.00,169.0,NaN,100.00,NaN,98.1,NaN,NaN
3,1,4.0,5.0,5.0,NaN,NaN,115.00,3.0,NaN,13.0,...,NaN,73.0,18.00,177.0,NaN,100.00,NaN,98.1,NaN,NaN
4,1,6.0,5.0,5.0,NaN,NaN,102.00,3.0,NaN,13.0,...,NaN,76.0,18.00,162.0,NaN,100.00,NaN,98.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1,239.0,6.0,5.0,NaN,NaN,65.00,4.0,NaN,15.0,...,NaN,89.0,16.00,110.0,NaN,99.00,NaN,99.5,NaN,NaN
76,1,240.0,NaN,NaN,97.0,0.5,NaN,NaN,103.0,NaN,...,3.7,NaN,NaN,NaN,135.0,NaN,24.5,NaN,6.0,14.99
77,1,241.0,6.0,5.0,NaN,NaN,80.00,4.0,NaN,15.0,...,NaN,89.0,17.00,127.0,NaN,99.00,NaN,97.5,NaN,NaN
78,1,247.0,6.0,5.0,NaN,NaN,68.25,4.0,NaN,15.0,...,NaN,88.5,16.75,113.5,NaN,97.25,NaN,97.7,NaN,NaN


In [5]:
def prepare_temporal(temporal_wide):
    temporal = []
    horizons = []
    for k, v in temporal_wide.groupby("id"):
        h = v["time"].values.tolist()
        local_data = v.drop(columns = ["id", "time"])
        local_data.index = h
        horizons.append(h)
        temporal.append(local_data.fillna(0))
    return temporal, horizons

train_temporal, train_horizons = prepare_temporal(train_temporal_wide)
test_temporal, test_horizons = prepare_temporal(test_temporal_wide)

train_outcome = train_static["icu_admission"]
test_outcome = test_static["icu_admission"]
train_static = train_static.drop(columns = ["id", "icu_admission"]).fillna(0)
test_static = test_static.drop(columns = ["id", "icu_admission"]).fillna(0)

In [6]:
assert len(train_temporal) == len(train_static)
assert len(test_temporal) == len(test_static)

In [7]:
from synthcity.plugins.core.dataloader import (
     TimeSeriesDataLoader,
)

dataloader_train = TimeSeriesDataLoader(
    temporal_data=train_temporal,
    temporal_horizons=train_horizons,
    static_data=train_static,
    outcome = train_outcome.to_frame(),
)


dataloader_test = TimeSeriesDataLoader(
    temporal_data=test_temporal,
    temporal_horizons=test_horizons,
    static_data=test_static,
    outcome = test_outcome.to_frame(),
)

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [ ]:
train_static_eval, train_temporal_eval, train_horizons_eval, train_outcome_eval = dataloader_train.unpack(as_numpy = True)

In [12]:
for idx, val in enumerate(train_temporal_eval):
    assert train_temporal_eval[idx].shape[0] == len(train_horizons_eval[idx]), len(train_horizons_eval[idx])

In [13]:
train_temporal[0]

variable,Best Motor Response,Best Verbal Response,CHLORIDE,CREATINEINE,DBP,Eye Opening,GLUCLOSE,Glasgow Coma Scale Score,HEMOGLOBIN,O2 Device: Aerosol mask,...,POTASSIUM,Pulse,Respiratory Rate,SBP,SODIUM,SpO2,TOTAL CO2,Temperature,UREA NITROGEN,WHITE BLOOD CELL COUNT
0.0,5.0,5.0,99.0,0.6,107.00,3.0,133.0,13.0,13.5,0.0,...,3.8,78.0,12.00,174.0,136.0,100.00,21.0,98.6,8.0,19.75
2.0,5.0,5.0,0.0,0.0,96.50,3.0,0.0,13.0,0.0,0.0,...,0.0,85.5,17.00,156.5,0.0,100.00,0.0,98.1,0.0,0.00
3.0,5.0,5.0,0.0,0.0,104.00,3.0,0.0,13.0,0.0,0.0,...,0.0,79.0,18.00,169.0,0.0,100.00,0.0,98.1,0.0,0.00
4.0,5.0,5.0,0.0,0.0,115.00,3.0,0.0,13.0,0.0,0.0,...,0.0,73.0,18.00,177.0,0.0,100.00,0.0,98.1,0.0,0.00
6.0,5.0,5.0,0.0,0.0,102.00,3.0,0.0,13.0,0.0,0.0,...,0.0,76.0,18.00,162.0,0.0,100.00,0.0,98.1,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239.0,6.0,5.0,0.0,0.0,65.00,4.0,0.0,15.0,0.0,0.0,...,0.0,89.0,16.00,110.0,0.0,99.00,0.0,99.5,0.0,0.00
240.0,0.0,0.0,97.0,0.5,0.00,0.0,103.0,0.0,12.3,0.0,...,3.7,0.0,0.00,0.0,135.0,0.00,24.5,0.0,6.0,14.99
241.0,6.0,5.0,0.0,0.0,80.00,4.0,0.0,15.0,0.0,0.0,...,0.0,89.0,17.00,127.0,0.0,99.00,0.0,97.5,0.0,0.00
247.0,6.0,5.0,0.0,0.0,68.25,4.0,0.0,15.0,0.0,0.0,...,0.0,88.5,16.75,113.5,0.0,97.25,0.0,97.7,0.0,0.00


In [1]:
from synthcity.plugins.core.models.ts_model import TimeSeriesModel

model = TimeSeriesModel(
        task_type = "classification",  # regression, classification
         n_static_units_in = train_static.shape[-1],
         n_temporal_units_in = train_temporal[0].shape[-1],
         n_temporal_window = max([len(t) for t in train_temporal]),
         output_shape = [2],
         mode = "RNN",
)



[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


NameError: name 'train_static' is not defined

In [ ]:
model.fit(train_static_eval, train_temporal_eval, train_horizons_eval, train_outcome_eval)

torch.Size([12, 151]) torch.Size([12, 49]) torch.Size([12, 102])
torch.Size([69, 151]) torch.Size([69, 49]) torch.Size([69, 102])
torch.Size([56, 151]) torch.Size([56, 49]) torch.Size([56, 102])
torch.Size([43, 151]) torch.Size([43, 49]) torch.Size([43, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([73, 151]) torch.Size([73, 49]) torch.Size([73, 102])
torch.Size([60, 151]) torch.Size([60, 49]) torch.Size([60, 102])
torch.Size([72, 151]) torch.Size([72, 49]) torch.Size([72, 102])
torch.Size([64, 151]) torch.Size([64, 49]) torch.Size([64, 102])
torch.Size([28, 151]) torch.Size([28, 49]) torch.Size([28, 102])
torch.Size([62, 151]) torch.Size([62, 49]) torch.Size([62, 102])
torch.Size([16, 151]) torch.Size([16, 49]) torch.Size([16, 102])
torch.Size([5, 151]) torch.Size([5, 49]) torch.Size([5, 102])
torch.Size([28, 151]) torch.Size([28, 49]) torch.Size([28, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([21, 151]) torch.Size([

torch.Size([5, 151]) torch.Size([5, 49]) torch.Size([5, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([10, 151]) torch.Size([10, 49]) torch.Size([10, 102])
torch.Size([12, 151]) torch.Size([12, 49]) torch.Size([12, 102])
torch.Size([3, 151]) torch.Size([3, 49]) torch.Size([3, 102])
torch.Size([28, 151]) torch.Size([28, 49]) torch.Size([28, 102])
torch.Size([5, 151]) torch.Size([5, 49]) torch.Size([5, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([3, 151]) torch.Size([3, 49]) torch.Size([3, 102])
torch.Size([8, 151]) torch.Size([8, 49]) torch.Size([8, 102])
torch.Size([5, 151]) torch.Size([5, 49]) torch.Size([5, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([8, 151]) torch.Size([8, 49]) torch.Size([8, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([8, 151]) torch.Size([8, 49]) torch.Size([8, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])

torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([14, 151]) torch.Size([14, 49]) torch.Size([14, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([20, 151]) torch.Size([20, 49]) torch.Size([20, 102])
torch.Size([17, 151]) torch.Size([17, 49]) torch.Size([17, 102])
torch.Size([11, 151]) torch.Size([11, 49]) torch.Size([11, 102])
torch.Size([12, 151]) torch.Size([12, 49]) torch.Size([12, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([5, 151]) torch.Size([5, 49]) torch.Size([5, 102])
torch.Size([7, 151]) torch.Size([7, 49]) torch.Size([7, 102])
torch.Size([11, 151]) torch.Size([11, 49]) torch.Size([11, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([3, 151]) torch.Size([3, 49]) torch.Size([3, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size(

torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Si

torch.Size([76, 151]) torch.Size([76, 49]) torch.Size([76, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([22, 151]) torch.Size([22, 49]) torch.Size([22, 102])
torch.Size([63, 151]) torch.Size([63, 49]) torch.Size([63, 102])
torch.Size([11, 151]) torch.Size([11, 49]) torch.Size([11, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([17, 151]) torch.Size([17, 49]) torch.Size([17, 102])
torch.Size([29, 151]) torch.Size([29, 49]) torch.Size([29, 102])
torch.Size([60, 151]) torch.Size([60, 49]) torch.Size([60, 102])
torch.Size([55, 151]) torch.Size([55, 49]) torch.Size([55, 102])
torch.Size([11, 151]) torch.Size([11, 49]) torch.Size([11, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([72, 151]) torch.Size([72, 49]) torch.Size([72, 102])
torch.Size([6, 151]) torch.Size([6, 49]) torch.Size([6, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([4, 151]) torch.Size([4, 49])

...

torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([7, 151]) torch.Size([7, 49]) torch.Size([7, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([3, 151]) torch.Size([3, 49]) torch.Size([3, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([3, 151]) torch.Size([3, 49]) torch.Size([3, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Si

torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Si

torch.Size([20, 151]) torch.Size([20, 49]) torch.Size([20, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([52, 151]) torch.Size([52, 49]) torch.Size([52, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([15, 151]) torch.Size([15, 49]) torch.Size([15, 102])
torch.Size([12, 151]) torch.Size([12, 49]) torch.Size([12, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([6, 151]) torch.Size([6, 49]) torch.Size([6, 102])
torch.Size([5, 151]) torch.Size([5, 49]) torch.Size([5, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([11, 151]) torch.Size([11, 49]) torch.Size([11, 102])
torch.Size([14, 151]) torch.Size([14, 49]) torch.Size([14, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([22, 151]) torch.Size([22, 49]) torch.Siz

torch.Size([40, 151]) torch.Size([40, 49]) torch.Size([40, 102])
torch.Size([10, 151]) torch.Size([10, 49]) torch.Size([10, 102])
torch.Size([61, 151]) torch.Size([61, 49]) torch.Size([61, 102])
torch.Size([43, 151]) torch.Size([43, 49]) torch.Size([43, 102])
torch.Size([8, 151]) torch.Size([8, 49]) torch.Size([8, 102])
torch.Size([52, 151]) torch.Size([52, 49]) torch.Size([52, 102])
torch.Size([27, 151]) torch.Size([27, 49]) torch.Size([27, 102])
torch.Size([76, 151]) torch.Size([76, 49]) torch.Size([76, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([22, 151]) torch.Size([22, 49]) torch.Size([22, 102])
torch.Size([63, 151]) torch.Size([63, 49]) torch.Size([63, 102])
torch.Size([11, 151]) torch.Size([11, 49]) torch.Size([11, 102])
torch.Size([4, 151]) torch.Size([4, 49]) torch.Size([4, 102])
torch.Size([17, 151]) torch.Size([17, 49]) torch.Size([17, 102])
torch.Size([29, 151]) torch.Size([29, 49]) torch.Size([29, 102])
torch.Size([60, 151]) torch.Size([

torch.Size([3, 151]) torch.Size([3, 49]) torch.Size([3, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
torch.Size([1, 151]) torch.Size([1, 49]) torch.Size([1, 102])
torch.Size([2, 151]) torch.Size([2, 49]) torch.Size([2, 102])
